In [129]:

# mount to google driver in order to use data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [130]:
# import section
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as  np
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.multiclass import OneVsRestClassifier

In [2]:
def show_res_count(y_hat):
  dir = {i:0 for i in range(6)}
  for i in y_hat:
    dir[i]+=1
  for i in dir:
    print(i,dir[i])

In [18]:
#Use satisfaction score , try to predict the answer
#Merge two dataframes and replace answer string to code

result = pd.merge(Train_IDs, status, on="Customer ID")
temp = satisfaction.fillna(2.5)
result = pd.merge(result , temp , on = "Customer ID")
replace_dir = {"Churn Category" : {"No Churn":0 , "Competitor":1 , "Dissatisfaction":2 , "Attitude" : 3 , "Price" : 4 , "Other" : 5}}
result = result.replace(replace_dir)
X = result["Satisfaction Score"]
Y = result['Churn Category']

In [20]:
num_data = [
 'Total Revenue',
 'Tenure in Months',
 'Number of Referrals',
 'Age',
 'Number of Dependents',
 'Longitude',
 'Avg Monthly GB Download',
 'Total Refunds',
 'Total Long Distance Charges',
 'Latitude',

 'Avg Monthly Long Distance Charges',
 'Monthly Charge',
 'Total Charges',
 'Total Extra Data Charges',

]

In [138]:
Test_IDs = pd.read_csv('/content/drive/MyDrive/HTML final project/Test_IDs.csv')
Train_IDs = pd.read_csv('/content/drive/MyDrive/HTML final project/Train_IDs.csv')
demographics = pd.read_csv('/content/drive/MyDrive/HTML final project/demographics.csv')
location = pd.read_csv('/content/drive/MyDrive/HTML final project/location.csv')
population = pd.read_csv('/content/drive/MyDrive/HTML final project/population.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/HTML final project/sample_submission.csv')
satisfaction = pd.read_csv('/content/drive/MyDrive/HTML final project/satisfaction.csv')
services = pd.read_csv('/content/drive/MyDrive/HTML final project/services.csv')
status = pd.read_csv('/content/drive/MyDrive/HTML final project/status.csv')
data = pd.read_csv('/content/drive/MyDrive/HTML final project/all_impute_data.csv')

In [139]:

all = [
  'Customer ID',
  'Churn Category',
  'Count',
  'Gender',
  'Age',
  'Under 30',
  'Senior Citizen',
  'Married',
  'Dependents',
  'Number of Dependents',
  'City',
  'Latitude',
  'Longitude',
  'Referred a Friend',
  'Number of Referrals',
  'Tenure in Months',
  'Offer',
  'Phone Service',
  'Avg Monthly Long Distance Charges',
  'Multiple Lines',
  'Internet Service',
  'Internet Type',
  'Avg Monthly GB Download',
  'Online Security',
  'Online Backup',
  'Device Protection Plan',
  'Premium Tech Support',
  'Streaming TV',
  'Streaming Movies',
  'Streaming Music',
  'Unlimited Data',
  'Contract',
  'Paperless Billing',
  'Payment Method',
  'Monthly Charge',
  'Total Charges',
  'Total Refunds',
  'Total Extra Data Charges',
  'Total Long Distance Charges',
  'Total Revenue',
  'Satisfaction Score',
]
human_knowledge  = data[all]

features = all[2:] + ["pro", "entertainment"]
human_knowledge  = data[all]
human_knowledge = human_knowledge.copy()

# Encoding

In [140]:
data

Unnamed: 0 Customer ID  ...  Satisfaction Score  Churn Category
0              0  1746-TGTWV  ...                 NaN               1
1              1  8499-BRXTD  ...                 3.0               1
2              2  2041-JIJCI  ...                 NaN               1
3              3  5832-EXGTT  ...                 2.0               2
4              4  8350-NYMVI  ...                 2.0               4
...          ...         ...  ...                 ...             ...
4221        4221  7817-BOQPW  ...                 5.0               0
4222        4222  4767-HZZHQ  ...                 3.0               0
4223        4223  6981-TDRFT  ...                 4.0               0
4224        4224  3094-JOJAI  ...                 5.0               0
4225        4225  9079-YEXQJ  ...                 4.0               0

[4226 rows x 48 columns]

In [143]:
human_knowledge["entertainment"] = np.zeros(len(human_knowledge))
human_knowledge["pro"] = np.zeros(len(human_knowledge))
# service

data["Streaming TV"] = data["Streaming TV"].fillna(0)
data["Streaming Movies"] = data["Streaming Movies"].fillna(0)
data["Streaming Music"] = data["Streaming Music"].fillna(0)

for i in ["Online Security",	"Online Backup"	,"Device Protection Plan",	"Premium Tech Support"]:
  data[i] = data[i].fillna(0)



for index, row in data.iterrows():
  human_knowledge.loc[index,"entertainment"] = row["Streaming TV"]*4 + row["Streaming Movies"]*2 + row["Streaming Music"]
  count = 1
  for i in ["Online Security",	"Online Backup"	,"Device Protection Plan",	"Premium Tech Support"]:
    human_knowledge.loc[index,"pro"] +=  row[i]*count
    count *= 2

human_knowledge = human_knowledge.dropna(subset=["Churn Category"])
human_knowledge = human_knowledge.dropna(thresh=3)

"""
# z-score 
for col in human_knowledge:
  if col != "Customer ID" and col != "Churn Category" and col in num_data:
    human_knowledge[col] = (human_knowledge[col] - human_knowledge[col].mean())/human_knowledge[col].std()
"""
"""
for i in human_knowledge:
  if i != "Customer ID" and i != "Churn Category" :
   # if i in num_data:
   #   human_knowledge[i] = human_knowledge[i].fillna(3)
    print(i)
   # else:
    #human_knowledge[i] = human_knowledge[i].fillna(human_knowledge[i].median())
    #human_knowledge[i] = human_knowledge[i].fillna(human_knowledge[i].mean())
    #human_knowledge[i] = human_knowledge[i].fillna(human_knowledge[i].mode()[0])
    #human_knowledge[i] = human_knowledge[i].fillna(3)
    
    human_knowledge[i] = KNN(k=3).fit_transform(human_knowledge[i].to_numpy().reshape(1,-1))
"""
#!pip install fancyimpute
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler


In [141]:
for i in human_knowledge:
  if i != "Customer ID" and i != "Churn Category" :
   # if i in num_data:
   #   human_knowledge[i] = human_knowledge[i].fillna(3)
    print(i)
   # else:
    #human_knowledge[i] = human_knowledge[i].fillna(human_knowledge[i].median())
    #human_knowledge[i] = human_knowledge[i].fillna(human_knowledge[i].mean())
    #human_knowledge[i] = human_knowledge[i].fillna(human_knowledge[i].mode()[0])
    #human_knowledge[i] = human_knowledge[i].fillna(3)
    


Count
Gender
Age
Under 30
Senior Citizen
Married
Dependents
Number of Dependents
City
Latitude
Longitude
Referred a Friend
Number of Referrals
Tenure in Months
Offer
Phone Service
Avg Monthly Long Distance Charges
Multiple Lines
Internet Service
Internet Type
Avg Monthly GB Download
Online Security
Online Backup
Device Protection Plan
Premium Tech Support
Streaming TV
Streaming Movies
Streaming Music
Unlimited Data
Contract
Paperless Billing
Payment Method
Monthly Charge
Total Charges
Total Refunds
Total Extra Data Charges
Total Long Distance Charges
Total Revenue
Satisfaction Score


Imputing row 1/1 with 0 missing, elapsed time: 0.000


/usr/local/lib/python3.7/dist-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


array([[1., 1., 1., ..., 1., 1., 1.]])

In [144]:
from sklearn.model_selection import cross_val_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
# for i in [ i for i in range(500,2000,100)]:
#   print(i)
# clf = svm.SVC(decision_function_shape='ovo')
# clf = svm.SVC(kernel='linear')
# clf = Pipeline(
#     [
#         # ("anova", SelectPercentile(chi2)),
#         ("scaler", StandardScaler()),
#         ("svc", SVC(gamma="auto")),
#     ]
# )
softImpute = SoftImpute()
#clf = SVC(kernel='linear', C=1, decision_function_shape='ovr')
clf = KNeighborsClassifier(n_neighbors=3)
X = softImpute.fit_transform(human_knowledge[features])

clf.fit(X, human_knowledge["Churn Category"])
sum(cross_val_score(clf, X,human_knowledge["Churn Category"], cv=10, scoring='f1_macro'))

[SoftImpute] Max Singular Value of X_init = 318618.414616
[SoftImpute] Iter 1: observed MAE=10.429201 rank=3
[SoftImpute] Iter 2: observed MAE=10.433016 rank=3
[SoftImpute] Iter 3: observed MAE=10.440636 rank=3
[SoftImpute] Iter 4: observed MAE=10.445382 rank=3
[SoftImpute] Iter 5: observed MAE=10.448445 rank=3
[SoftImpute] Iter 6: observed MAE=10.450371 rank=3
[SoftImpute] Iter 7: observed MAE=10.451579 rank=3
[SoftImpute] Iter 8: observed MAE=10.452222 rank=3
[SoftImpute] Iter 9: observed MAE=10.452528 rank=3
[SoftImpute] Iter 10: observed MAE=10.452659 rank=3
[SoftImpute] Iter 11: observed MAE=10.452698 rank=3
[SoftImpute] Iter 12: observed MAE=10.452680 rank=3
[SoftImpute] Iter 13: observed MAE=10.452646 rank=3
[SoftImpute] Iter 14: observed MAE=10.452603 rank=3
[SoftImpute] Iter 15: observed MAE=10.452549 rank=3
[SoftImpute] Iter 16: observed MAE=10.452498 rank=3
[SoftImpute] Iter 17: observed MAE=10.452456 rank=3
[SoftImpute] Iter 18: observed MAE=10.452426 rank=3
[SoftImpute] It

1.805081396226757

In [111]:
sum(np.isnan(X))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
1.412908208518382

In [33]:
test = pd.merge(Test_IDs["Customer ID"].to_frame(), satisfaction , on = "Customer ID" , how ='left')
test = pd.merge(test ,services ,on = "Customer ID" , how ='left')
test = pd.merge(test ,demographics,on = "Customer ID" , how ='left')
test = pd.merge(test ,location,on = "Customer ID" , how ='left')

test['Streaming TV'].replace(['No', 'Yes'], [0, 1], inplace = True)
test['Streaming Movies'].replace(['No', 'Yes'], [0, 1], inplace = True)
test['Streaming Music'].replace(['No', 'Yes'], [0, 1], inplace = True)
test['Offer'].replace(sorted(services['Offer'].loc[~services['Offer'].isnull()].unique()),
                          list(range(len(services['Offer'].loc[~services['Offer'].isnull()].unique()))), inplace = True)
test['Payment Method'].replace(sorted(services['Payment Method'].loc[~services['Payment Method'].isnull()].unique()),
                         list(range(len(services['Payment Method'].loc[~services['Payment Method'].isnull()].unique()))), inplace = True)
test['Online Security'].replace(['No', 'Yes'], [0, 1], inplace = True)
test['Online Backup'].replace(['No', 'Yes'], [0, 1], inplace = True)
test['Device Protection Plan'].replace(['No', 'Yes'], [0, 1], inplace = True)
test['Premium Tech Support'].replace(['No', 'Yes'], [0, 1], inplace = True)
test['Married'].replace(['No', 'Yes'], [0, 1],inplace = True)
test['Dependents'].replace(['No', 'Yes'], [0, 1],inplace = True)
test["Streaming TV"] = test["Streaming TV"].fillna(0)
test["Streaming Movies"] = test["Streaming Movies"].fillna(0)
test["Streaming Music"] = test["Streaming Music"].fillna(0)
test['Senior Citizen'].replace(['No', 'Yes'], [0, 1],inplace = True)
test['Gender'].replace(['Female', 'Male'], [0, 1],inplace = True)
test['Under 30'].replace(['No', 'Yes'], [0, 1],inplace = True)
for i in ["Online Security",	"Online Backup"	,"Device Protection Plan",	"Premium Tech Support"]:
  test[i] = test[i].fillna(0)
#test['Device Protection Plan'].replace(['No', 'Yes'], [0, 1], inplace = True)
#test['Premium Tech Support'].replace(['No', 'Yes'], [0, 1], inplace = True)
test['Unlimited Data'].replace(['No', 'Yes'], [0, 1], inplace = True)
test['Paperless Billing'].replace(['No', 'Yes'], [0, 1], inplace = True)
test['Contract'].replace(sorted(services['Contract'].loc[~services['Contract'].isnull()].unique()),
                         list(range(len(services['Contract'].loc[~services['Contract'].isnull()].unique()))), inplace = True)
#test['Offer'].replace(sorted(services['Offer'].loc[~services['Offer'].isnull()].unique()),
 #                        list(range(len(services['Offer'].loc[~services['Offer'].isnull()].unique()))), inplace = True)

test['Referred a Friend'].replace(['No', 'Yes'], [0, 1], inplace = True)
test['Phone Service'].replace(['No', 'Yes'], [0, 1], inplace = True)
test['Multiple Lines'].replace(['No', 'Yes'], [0, 1], inplace = True)
#test['Internet Service'].replace(['No', 'Yes'], [0, 1], inplace = True)
test['Internet Type'].replace(sorted(services['Internet Type'].loc[~services['Internet Type'].isnull()].unique()),
                         list(range(len(services['Internet Type'].loc[~services['Internet Type'].isnull()].unique()))), inplace = True)
test["pro"] = np.zeros(len(test))
test["entertainment"] = np.zeros(len(test))
for index, row in test.iterrows():
  test.loc[index , "entertainment"] = row["Streaming TV"]*4 + row["Streaming Movies"]*2 + row["Streaming Music"]
  count = 1
  for i in ["Online Security",	"Online Backup"	,"Device Protection Plan",	"Premium Tech Support"]:
    test.loc[index , "pro"] +=  row[i]*count
    count *= 2
test['Internet Service'].replace(['No', 'Yes'], [0, 1], inplace = True)
test = test[features]
test['City'].replace(sorted(location['City'].loc[~location['City'].isnull()].unique()),
                        list(range(len(location['City'].loc[~location['City'].isnull()].unique()))), inplace = True)

"""
for col in test:
  if col in num_data:
    test[col] = (test[col] - test[col].mean())/test[col].std()
"""
for i in test:
  if i != "Customer ID" and i != "Churn Category":
    #if i in num_data:
    #  test[i] = test[i].fillna(3)
    #else:
      #test[i] = test[i].fillna(test[i].median())
      #test[i] = test[i].fillna(test[i].mean())
      test[i] = test[i].fillna(test[i].mode())




/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
#features= ["Total Revenue" , "Satisfaction Score" ,"Tenure in Months", "Age","Married" ,"Number of Dependents" , "Offer" , "Internet Service","Payment Method" , "Longitude" , "pro" , "entertainment"]
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsOneClassifier

estimators = [
    ('lgbm' ,LGBMClassifier(n_estimators=5000) ),
    ('xgb', XGBClassifier()),
   # ('cat', CatBoostClassifier())
]
#clf = StackingClassifier(
#   estimators=estimators, final_estimator=LGBMClassifier())
#clf = CatBoostClassifier()
#clf = XGBClassifier()
clf = LGBMClassifier(num_leaves=10 , max_depth=9 ) 
clf.fit(human_knowledge[features], human_knowledge["Churn Category"])


NameError: ignored

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
feature_selection = ExtraTreesClassifier(n_estimators=50)
feature_selection = feature_selection.fit(human_knowledge[features], human_knowledge["Churn Category"])

model = SelectFromModel(feature_selection, prefit=True)

X_new = model.transform(human_knowledge[features])

# SVM

In [ ]:
from sklearn import svm

In [ ]:


y_hat_svm =  clf.predict(test[features])
y_hat_svm

In [ ]:
show_res_count(y_hat_svm)

In [ ]:
from sklearn.svm import LinearSVC

clf2 = LinearSVC(random_state=1, tol=1e-5)

clf2.fit(human_knowledge[features], human_knowledge["Churn Category"])
y_hat_svm2 =  clf2.predict(test[features])
show_res_count(y_hat_svm2)

In [ ]:
svm_clf = SVC(C=6.7046885962229785, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.004147722987833689,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001)
svm_clf.fit(human_knowledge[features], human_knowledge["Churn Category"])
y_hat_svm_clf =  svm_clf.predict(test[features])
show_res_count(y_hat_svm_clf)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

In [ ]:
from scipy import stats
svm_clf2 = SVC(gamma="scale")
# param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}

param_distributions = {'C': stats.expon(scale=100), 'gamma': stats.expon(scale=.1),
  'kernel': ['rbf'], 'class_weight':['balanced', None]}
rnd_search_cv2 = RandomizedSearchCV(svm_clf2, param_distributions, n_iter=100, verbose=2, cv=5)
rnd_search_cv2.fit(human_knowledge[features], human_knowledge["Churn Category"])

In [ ]:
rnd_search_cv2.best_estimator_

In [ ]:
from sklearn.model_selection import cross_val_score
s = 0
scores = cross_val_score(rnd_search_cv2.best_estimator_, human_knowledge[features], human_knowledge["Churn Category"], cv=5)
s+=sum(scores)
s

In [ ]:
y_hat_svm_rnd_search_cv = rnd_search_cv.best_estimator_.predict(test[features])
show_res_count(y_hat_svm_rnd_search_cv)


In [ ]:

t = sample_submission["Customer ID"].to_frame()


t["Churn Category"]= y_hat_svm.astype(int)
t = t.fillna(0)
t
t.to_csv("svm_ans.csv", index=False)